<a href="https://colab.research.google.com/github/abhiskp/Pneumonia-Detection-in-Children/blob/main/CNN_multple_ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import classification_report, roc_auc_score


In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"abhiskp","key":"eed4e593d559cc8f4b1d011a8f33db10"}'}

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!kaggle datasets download -d andrewmvd/pediatric-pneumonia-chest-xray

pediatric-pneumonia-chest-xray.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip pediatric-pneumonia-chest-xray.zip

In [ ]:
    # train_dir = './Pediatric Chest X-ray Pneumonia/train'
    # test_dir = './Pediatric Chest X-ray Pneumonia/test'


In [ ]:
input_shape = (224, 224, 3)  # shape of input image

model = keras.Sequential(
    [
        layers.Conv2D(16, kernel_size=(3, 3), activation="relu", input_shape=input_shape),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dropout(0.5),
        layers.Dense(1, activation="sigmoid"),
    ]
)


In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    "./Pediatric Chest X-ray Pneumonia/train",
    target_size=input_shape[:2],
    batch_size=32,
    class_mode="binary",
    subset="training"
)

validation_generator = train_datagen.flow_from_directory(
    "./Pediatric Chest X-ray Pneumonia/test",
    target_size=input_shape[:2],
    batch_size=32,
    class_mode="binary",
    subset="validation"
)


Found 4187 images belonging to 2 classes.
Found 124 images belonging to 2 classes.


In [ ]:
epochs = 10

model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
)


Epoch 1/10
130/130 [==============================] - 286s 2s/step - loss: 0.2772 - accuracy: 0.8828 - val_loss: 0.8742 - val_accuracy: 0.6771
Epoch 2/10
130/130 [==============================] - 270s 2s/step - loss: 0.1366 - accuracy: 0.9519 - val_loss: 0.6420 - val_accuracy: 0.7396
Epoch 3/10
130/130 [==============================] - 276s 2s/step - loss: 0.1052 - accuracy: 0.9620 - val_loss: 1.0026 - val_accuracy: 0.6979
Epoch 4/10
130/130 [==============================] - 269s 2s/step - loss: 0.0874 - accuracy: 0.9661 - val_loss: 0.9211 - val_accuracy: 0.7188
Epoch 5/10
130/130 [==============================] - 277s 2s/step - loss: 0.0752 - accuracy: 0.9714 - val_loss: 0.5662 - val_accuracy: 0.7708
Epoch 6/10
130/130 [==============================] - 301s 2s/step - loss: 0.0664 - accuracy: 0.9767 - val_loss: 1.0518 - val_accuracy: 0.7083
Epoch 7/10
130/130 [==============================] - 285s 2s/step - loss: 0.0498 - accuracy: 0.9827 - val_loss: 1.6703 - val_accuracy: 0.7188

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    "./Pediatric Chest X-ray Pneumonia/train",
    target_size=input_shape[:2],
    batch_size=32,
    class_mode="binary"
    )

y_true = test_generator.classes

y_pred = model.predict(test_generator)
y_pred = y_pred.flatten()

print("Classification Report:")
print(classification_report(y_true, y_pred > 0.5))

print("AUC Score:", roc_auc_score(y_true, y_pred))


Found 5232 images belonging to 2 classes.
164/164 [==============================] - 148s 899ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.27      0.27      0.27      1349
           1       0.75      0.75      0.75      3883

    accuracy                           0.62      5232
   macro avg       0.51      0.51      0.51      5232
weighted avg       0.62      0.62      0.62      5232

AUC Score: 0.5013207864506802


In [ ]:
# y_pred = model.predict('./Pediatric Chest X-ray Pneumonia/test')
# y_pred_binary = np.round(y_pred)
# tn, fp, fn, tp = confusion_matrix(, y_pred_binary).ravel()


#  ResNet with CNN

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
from sklearn.metrics import confusion_matrix, f1_score
import numpy as np

In [ ]:
# Define the ResNet50 model
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the weights of the ResNet50 model
for layer in resnet_model.layers:
    layer.trainable = False

# Define the CNN model
input_tensor = Input(shape=(224, 224, 3))
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_tensor)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)
cnn_model = Model(inputs=input_tensor, outputs=x)

# Combine the ResNet50 and CNN models
combined_model = Model(inputs=resnet_model.input, outputs=cnn_model(resnet_model.output))

# Compile the model
combined_model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['acc', AUC()])



94765736/94765736 [==============================] - 1s 0us/step


ValueError: ignored

In [ ]:
# Train the model
combined_model.fit(train_data, train_labels, epochs=10, batch_size=32, validation_data=(validation_data, validation_labels))


In [ ]:



# Test the model
y_pred = combined_model.predict(test_data)
y_pred_binary = np.round(y_pred)
tn, fp, fn, tp = confusion_matrix(test_labels, y_pred_binary).ravel()
accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1 = f1_score(test_labels, y_pred_binary)
auc = roc_auc_score(test_labels, y_pred)

print("True Positive: ", tp)
print("True Negative: ", tn)
print("F1 Score: ", f1)
print("AUC: ", auc)


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet import ResNet50

# Define the input shape
input_shape = (224, 224, 3)

# Create the ResNet50 model
resnet = ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)

# Freeze the ResNet50 layers
for layer in resnet.layers:
    layer.trainable = False

# Add a Faster CNN network on top of ResNet50
x = resnet.output
x = Conv2D(32, kernel_size=(3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

# Combine the ResNet50 and Faster CNN models
model = Model(inputs=resnet.input, outputs=predictions)

# Compile the model with binary crossentropy loss and Adam optimizer
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define image data preprocessing
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Load and preprocess the training and validation data
train_data = train_datagen.flow_from_directory(
    './Pediatric Chest X-ray Pneumonia/train',
    target_size=input_shape[:2],
    batch_size=32,
    class_mode='binary')

validation_data = validation_datagen.flow_from_directory(
    './Pediatric Chest X-ray Pneumonia/test',
    target_size=input_shape[:2],
    batch_size=32,
    class_mode='binary')

# Train the model
model.fit(
    train_data,
    epochs=10,
    validation_data=validation_data,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=2)])

# Evaluate the model
results = model.evaluate(validation_data)
print('Validation Loss:', results[0])
print('Validation Accuracy:', results[1])

# Generate predictions on the test set
test_data = validation_datagen.flow_from_directory(
    './Pediatric Chest X-ray Pneumonia/test',
    target_size=input_shape[:2],
    batch_size=32,
    class_mode='binary')

y_pred = model.predict(test_data)
y_true = test_data.classes

# Calculate true positives, true negatives, false positives, and false negatives
tp = tf.keras.metrics.TruePositives()(y_true, y_pred)
tn = tf.keras.metrics.TrueNegatives()(y_true, y_pred)
fp = tf.keras.metrics.FalsePositives()(y_true, y_pred)
fn = tf.keras.metrics.FalseNegatives()(y_true, y_pred)

# Calculate F1 score and AUC
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2 * precision * recall / (precision + recall)
auc = tf.keras.metrics.AUC()(y_true, y_pred)

Found 5232 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Epoch 1/10
164/164 [==============================] - 1275s 8s/step - loss: 0.5769 - accuracy: 0.7366 - val_loss: 0.6240 - val_accuracy: 0.6250
Epoch 2/10
164/164 [==============================] - 1257s 8s/step - loss: 0.4676 - accuracy: 0.7521 - val_loss: 0.5355 - val_accuracy: 0.7404
Epoch 3/10
164/164 [==============================] - 1254s 8s/step - loss: 0.4216 - accuracy: 0.7978 - val_loss: 0.5500 - val_accuracy: 0.7372
Epoch 4/10
20/20 [==============================] - 120s 6s/step - loss: 0.6044 - accuracy: 0.7244
Validation Loss: 0.6044395565986633
Validation Accuracy: 0.7243589758872986
Found 624 images belonging to 2 classes.
20/20 [==============================] - 121s 6s/step


In [ ]:
rint('True Positives:', tp)
print('True Negatives:', tn)
print('False Positives:', fp)
print('False Negatives:', fn)
print('F1 Score:', f1_score)
print('AUC:', auc)

NameError: ignored